<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex10/Alexander_Valle/Alexander_Valle_IA025_Aula10_Exerc%C3%ADcio_44M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = 'Rolan Alexander Valle Rey Sánchez ra230254'
print(f'Meu nome é {nome}')

Meu nome é Rolan Alexander Valle Rey Sánchez ra230254


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 75.5 MB/s 
     |████████████████████████████████| 6.6 MB 63.8 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [ ]:
# Check which GPU we are using
!nvidia-smi

Wed Jun  8 20:18:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


In [ ]:
class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        self.tokens_ids =[]
        starid=tokenize(text='[CLS]', tokenizer=tokenizer)        #print('starid',starid)
        padid=tokenizer.pad_token_id        #print('padid',padid)
        for text in texts:
          #print('text: ',text)
          tokens = tokenize(text, tokenizer)          #print(len(tokens),max_seq_length)
          if len(tokens) <= max_seq_length:
            self.tokens_ids.append(starid+tokens[0:len(tokens)]+[ padid for k in range(len(tokens),max_seq_length)])
          else:
            for i in range(len(tokens)+1 - max_seq_length):
              self.tokens_ids.append(starid+tokens[i:i +max_seq_length])
        self.tokens_ids=torch.tensor(self.tokens_ids, dtype=torch.long)
    def __len__(self):
        return len(self.tokens_ids)
    def __getitem__(self, idx):
        return self.tokens_ids[idx,:-1], self.tokens_ids[idx,1:]

## Testando se a implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

In [ ]:
dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)


In [ ]:
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')


Passou no assert de tamanho do dataset.


In [ ]:
first_batch_input, first_batch_target = next(iter(dummy_loader))
first_batch_input, first_batch_target

(tensor([[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
         [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]]),
 tensor([[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
         [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]]))

In [ ]:
first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

Passou no assert de dataset.


In [ ]:
len(dummy_dataset)

2

In [ ]:
#test with log text and short text

dummy_texts = ['Eu gosto de correr e de comer muita pizza de carne', 'Ela gosta muito de comer pizza']
dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,  125, 13239,  122,    125,  1847,  5747],
     [  101,  10303, 125, 13239,   122,  125,   1847,  5747, 13779],
     [  101,   125, 13239,   122,   125,  1847,  5747, 13779, 15616],
    [  101, 13239,   122,   125,  1847,  5747, 13779, 15616,   125],
     [  101,  1660,  5971,  785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,  125, 13239,  122,    125,  1847,  5747, 13779],
     [ 10303, 125, 13239,   122,  125,   1847,  5747, 13779, 15616],
     [  125, 13239,   122,   125,  1847,  5747, 13779, 15616,   125],
    [13239,   122,   125,  1847,  5747, 13779, 15616,   125,  7714],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de dataset.')


Passou no assert de tamanho do dataset.
Passou no assert de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

File ‘sample-1gb.txt’ already there; not retrieving.



In [ ]:
# Load datasets
max_seq_length = 9

train_examples = 40000
valid_examples = 2000
test_examples = 4000

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.
Truncating to 46000 lines.


In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 44099473
valid examples: 2113552
test examples: 4220703


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
  #https://github.com/happy-jihye/Natural-Language-Processing/tree/main/code
    def __init__(self, hid_dim, n_heads):
        super().__init__()
        assert hid_dim % n_heads == 0
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        self.Wq = nn.Linear(hid_dim, hid_dim)
        self.Wk = nn.Linear(hid_dim, hid_dim)
        self.Wv = nn.Linear(hid_dim, hid_dim)
        self.Wo = nn.Linear(hid_dim, hid_dim)
        self.dropout = nn.Dropout(0.1)
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        B = query.shape[0]#batch_size
        Q = self.Wq(query)#query: Q = [batch size, L, hid_dim]
        K = self.Wk(key)#key :K = [batch size, key len, hid_dim]
        V = self.Wv(value)#value : V = [batch size, value len, hid_dim]
        Q = Q.view(B, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)#Q = [B, n_heads, L, head dim]
        K = K.view(B, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)#K = [B, n_heads, key len, head dim]
        V = V.view(B, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)#V = [B, n_heads, value len, head dim]
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale#energy = [B, n_heads, L, key len]
        #if mask is not None:
        #    energy = energy.masked_fill(mask == 0, -1e10)
        attention = torch.softmax(energy, dim = -1)#attention = [B, n_heads, L, key len]
        x = torch.matmul(self.dropout(attention), V)#x = [B, n_heads, L, head dim]
        x = x.permute(0, 2, 1, 3).contiguous()#x = [B, L, n heads, head dim]
        x = x.view(B, -1, self.hid_dim)#x = [B, L, hid_dim]
        x = self.Wo(x)#x = [B, L, hid_dim]
        return x#, attention

In [ ]:
class TransfomerBlock(nn.Module):
    def __init__(self, input_dim,num_heads):
        """
        Args:
            input_dim: Dimensionality of the input
            num_heads: Number of heads to use in the attention block
            embed_dim: embedings 
        """
        super().__init__()
        self.self_attn = MultiHeadAttentionLayer( input_dim, num_heads)# Attention layer
        hsf=2#hidden size factor
        feedforward_dim=hsf*input_dim
        self.L1 = nn.Linear(input_dim, feedforward_dim)
        self.Drop1 =nn.Dropout(0.1)
        self.Drop2 =nn.Dropout(0.1)
        self.relu = nn.ReLU(inplace=True)#inplace=Truevocab_size
        self.L2 = nn.Linear(feedforward_dim,input_dim)#, bias=False
        # Two-layer MLP# Maps the final output sequence to class logits
        self.ff = nn.Sequential(self.L1 ,self.Drop1,self.relu,self.L2,self.Drop2)#feedforward
        self.debug=False
        self.norm1 = nn.LayerNorm(input_dim)
        self.norm2 = nn.LayerNorm(input_dim)
    def forward(self, x, mask=None):
        x = x + self.self_attn(x,x,x, mask=mask) #residual+Attention part
        E = self.norm1(x)
        fedforward = self.ff(E) # MLP part: B, V  
        out=self.norm2(fedforward + E)
        if self.debug:
          print('x.shape:', x.shape)
          #print('A.shape:', A.shape)
          print('E.shape',E.shape)
          print('FF.shape',fedforward.shape)
          print('out.shape',out.shape)
        return out
    

In [ ]:
class LanguageModel(torch.nn.Module):
    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """Implements the Self-attention, decoder-only.
        Args: vocab_size (int): Size of the input vocabulary.
              max_seq_length (int): Size of the sequence to consider as context for prediction.
              dim (int): Dimension of the embedding layer for each word in the context.
              n_layers (int): number of self-attention layers.
              pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        super().__init__()
        self.D=dim#embedding_dim  
        self.n_layes=n_layers
        self.nh=4# four heads
        self.L=max_seq_length
        self.pad_token_id=pad_token_id
        self.max_seq_length = max_seq_length
        self.C= nn.Embedding(vocab_size, dim, padding_idx=pad_token_id)#word embeddings
        self.P= nn.Embedding(max_seq_length, dim)# positional_embeddings of the words

        # Multi-Head Attention layes
        self.attention_layers = nn.ModuleList([TransfomerBlock( self.D, self.nh) for _ in range(n_layers)])
        self.l1=torch.nn.Linear(dim, 2*dim)
        self.relu=torch.nn.ReLU()
        self.l2=torch.nn.Linear(dim*2, vocab_size, bias=False)
        self.tologits = torch.nn.Sequential(self.l1,self.relu,self.l2)

    def get_mask(self, inputs):
        #https://github.com/bentrevett/pytorch-seq2seq/blob/master/6%20-%20Attention%20is%20All%20You%20Need.ipynb
        #= [batch size, L]
        B = inputs.shape[1]#batch size
        pad_mask = (inputs != self.pad_token_id).unsqueeze(1).unsqueeze(2)  #pad_mask = [B, 1, 1, L]
        sub_mask = torch.tril(torch.ones((self.L, self.L))).bool().to(device)   #sub_mask = [B, L]
        mask = pad_mask & sub_mask      #mask = [B, 1, L, L]
        return mask 

    def forward(self, inputs):
        """
        Args:
        inputs is a LongTensor of shape (batch_size, max_seq_length)
        Returns:
        logits of shape (batch_size, max_seq_length, vocab_size)
        """
        B = inputs.shape[0]# B: batch_size
        X=self.C(inputs) +self.P.weight# token embeddings +  position embeddings , L: max_seq_length   # shape = B, L, D
        mask=self.get_mask(inputs)
        for layer in self.attention_layers:
          # X shape: (B, L, D)
          X = layer(X, mask = mask)
        logits = self.tologits(X) # B, V # shape = B, L, D
        return logits

## Teste o modelo com um exemplo

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

In [ ]:
sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 5796288


## Assert da Perplexidade


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.
    Args:        logits: a FloatTensor of shape (batch_size, seq_len, vocab_size)
                  target: a LongTensor of shape (batch_size, seq_len)
    Returns:      A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)

n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30710
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
#max_examples =    150_000_000
#training examples: 18 268 128
max_examples = 10_000_000

eval_every_steps = 1000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 30911.82, valid ppl: 30532.46
1000 steps; 1024000 examples so far; train ppl: 86.85, valid ppl: 3.92
2000 steps; 2048000 examples so far; train ppl: 2.71, valid ppl: 2.34
3000 steps; 3072000 examples so far; train ppl: 2.29, valid ppl: 2.24
4000 steps; 4096000 examples so far; train ppl: 2.23, valid ppl: 2.20
5000 steps; 5120000 examples so far; train ppl: 2.19, valid ppl: 2.16
6000 steps; 6144000 examples so far; train ppl: 2.15, valid ppl: 2.13
7000 steps; 7168000 examples so far; train ppl: 2.13, valid ppl: 2.11
8000 steps; 8192000 examples so far; train ppl: 2.10, valid ppl: 2.09
9000 steps; 9216000 examples so far; train ppl: 2.08, valid ppl: 2.07


## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 2.0485470222335063


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Ontem fui ao restaurante comer um prato delicioso, pedi'#  tem medo de água fria
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Ontem fui ao restaurante comer um prato delicioso, pediilar
Ontem fui ao restaurante comer um prato delicioso, pediilar?
Ontem fui ao restaurante comer um prato delicioso, pediilar? se
Ontem fui ao restaurante comer um prato delicioso, pediilar? se mas
Ontem fui ao restaurante comer um prato delicioso, pediilar? se mas enquanto
Ontem fui ao restaurante comer um prato delicioso, pediilar? se mas enquanto perto
Ontem fui ao restaurante comer um prato delicioso, pediilar? se mas enquanto perto,
Ontem fui ao restaurante comer um prato delicioso, pediilar? se mas enquanto perto,.
Ontem fui ao restaurante comer um prato delicioso, pediilar? se mas enquanto perto,..
Ontem fui ao restaurante comer um prato delicioso, pediilar? se mas enquanto perto,...
Ontem fui ao restaurante comer um prato delicioso, pediilar? se mas enquanto perto,...?
Ontem fui ao restaurante comer um prato delicioso, pediilar? se mas enquanto perto,...? se
Ontem fui ao restaurante comer um prato delicioso, pediilar? se ma

In [ ]:
prompt = 'A grama da vizinho é sempre é sempre mais verde'# 
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

A grama da vizinho é sempre é sempre mais verdelas
A grama da vizinho é sempre é sempre mais verdelas da
A grama da vizinho é sempre é sempre mais verdelas da conforme
A grama da vizinho é sempre é sempre mais verdelas da conforme é
A grama da vizinho é sempre é sempre mais verdelas da conforme é sempre
A grama da vizinho é sempre é sempre mais verdelas da conforme é sempre é
A grama da vizinho é sempre é sempre mais verdelas da conforme é sempre é sempre
A grama da vizinho é sempre é sempre mais verdelas da conforme é sempre é sempre las
A grama da vizinho é sempre é sempre mais verdelas da conforme é sempre é sempre las -
A grama da vizinho é sempre é sempre mais verdelas da conforme é sempre é sempre las - ide
A grama da vizinho é sempre é sempre mais verdelas da conforme é sempre é sempre las - ide.
A grama da vizinho é sempre é sempre mais verdelas da conforme é sempre é sempre las - ide. conforme
A grama da vizinho é sempre é sempre mais verdelas da conforme é sempre é sempre las

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz se
Eu gosto de comer pizza pois me faz se com
Eu gosto de comer pizza pois me faz se com de
Eu gosto de comer pizza pois me faz se com de?
Eu gosto de comer pizza pois me faz se com de? las
Eu gosto de comer pizza pois me faz se com de? las geralmente
Eu gosto de comer pizza pois me faz se com de? las geralmente pois
Eu gosto de comer pizza pois me faz se com de? las geralmente pois me
Eu gosto de comer pizza pois me faz se com de? las geralmente pois me tem
Eu gosto de comer pizza pois me faz se com de? las geralmente pois me tem se
Eu gosto de comer pizza pois me faz se com de? las geralmente pois me tem se com
Eu gosto de comer pizza pois me faz se com de? las geralmente pois me tem se com de
Eu gosto de comer pizza pois me faz se com de? las geralmente pois me tem se com de?
Eu gosto de comer pizza pois me faz se com de? las geralmente pois me tem se com de? las
Eu gosto de comer pizza pois me faz se com de? las geralmente pois me tem se com de? 

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.